In [163]:
import numpy as np
import keras
import cv2
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
import itertools
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [164]:
file_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\img'
labels_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\labels.csv'

In [165]:
dftr = pd.read_csv(labels_path)

dftr["smiling"] = dftr["smiling"].replace(to_replace=[-1], value=['Frown'])
dftr["smiling"] = dftr["smiling"].replace(to_replace=[1], value=['Smile'])
dftr["gender"] = dftr["gender"].replace(to_replace=[-1], value=['Female'])
dftr["gender"] = dftr["gender"].replace(to_replace=[1], value=['Male'])

one_hot_s = pd.get_dummies(dftr["smiling"])
one_hot_g = pd.get_dummies(dftr["gender"])
dftr = dftr.drop(columns=["gender", "smiling"])
dftr = dftr.join(one_hot_g)
dftr = dftr.join(one_hot_s)

# dfg = pd.DataFrame(dftr["Female"])
# dfg = dfg.join(dftr["Male"])
# records = df.to_records(index=False)
# dfg = pd.DataFrame(records)

# dfs = pd.DataFrame(dftr["Frown"])
# dfs = dfs.join(dftr["Smile"])
# records = df.to_records(index=False)
# dfs = pd.DataFrame(records)

# dftr = dftr.drop(columns=["Female", "Male", "Frown", "Smile"])
# dftr = dftr.join(dfg)
# dftr = dftr.rename(columns={"0": "Gender", "Unnamed: 0": "Index"})
# dftr = dftr.join(dfs)
# dftr = dftr.rename(columns={"0": "Smiling"})

In [166]:
dftr

,Unnamed: 0,img_name,Female,Male,Frown,Smile
0,0,0.jpg,1,0,0,1
1,1,1.jpg,1,0,0,1
2,2,2.jpg,0,1,1,0
3,3,3.jpg,1,0,1,0
4,4,4.jpg,1,0,1,0
...,...,...,...,...,...,...
4995,4995,4995.jpg,0,1,0,1
4996,4996,4996.jpg,0,1,0,1
4997,4997,4997.jpg,0,1,0,1
4998,4998,4998.jpg,0,1,0,1


In [167]:
train, validation, test = \
              np.split(dftr.sample(frac=1), 
                       [int(.6*len(dftr)), int(.8*len(dftr))])

In [168]:
validation

,Unnamed: 0,img_name,Female,Male,Frown,Smile
1128,1128,1128.jpg,1,0,1,0
1794,1794,1794.jpg,1,0,0,1
4723,4723,4723.jpg,0,1,0,1
3298,3298,3298.jpg,0,1,1,0
4600,4600,4600.jpg,0,1,0,1
...,...,...,...,...,...,...
212,212,212.jpg,0,1,1,0
3175,3175,3175.jpg,0,1,1,0
3127,3127,3127.jpg,0,1,1,0
4651,4651,4651.jpg,0,1,0,1


In [169]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(dataframe=train, 
                                              directory=file_path, 
                                              x_col="img_name", 
                                              y_col=["Female", "Male"], 
                                              class_mode="raw", 
                                              target_size=(55,45), 
                                              batch_size=30, 
                                              color_mode='grayscale', 
                                              interpolation='bicubic')

validation_generator = datagen.flow_from_dataframe(dataframe=validation, 
                                              directory=file_path, 
                                              x_col="img_name", 
                                              y_col=["Female", "Male"], 
                                              class_mode="raw", 
                                              target_size=(55,45), 
                                              batch_size=10, 
                                              color_mode='grayscale', 
                                              interpolation='bicubic')

test_generator = datagen.flow_from_dataframe(dataframe=test, 
                                              directory=file_path, 
                                              x_col="img_name", 
                                              y_col=["Female", "Male"], 
                                              class_mode="raw", 
                                              target_size=(55,45), 
                                              batch_size=10, 
                                              color_mode='grayscale', 
                                              interpolation='bicubic')

Found 3000 validated image filenames.
Found 1000 validated image filenames.
Found 1000 validated image filenames.


In [170]:
# model = Sequential()
# model.add(Dense(32, activation='relu', input_shape=(55,45,1)))
# model.add(Flatten())
# model.add(Dense(1, activation=('softmax')))

model = Sequential()
model.add(InputLayer(input_shape=(55,45,1)))
model.add(Conv2D(filters=96, kernel_size=(4,4), strides=4, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=1, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(2,2), activation="relu", strides=1, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

In [171]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 14, 12, 96)        1632      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 6, 96)          0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 7, 6, 96)          384       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 7, 6, 256)         221440    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 3, 3, 256)         1024      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 3, 3, 256)       

In [172]:
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [173]:
train_steps = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

In [174]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=train_steps,
                    validation_data=validation_generator,
                    validation_steps=validation_steps,
                    epochs=5)

Epoch 1/5
100/100 [==============================] - 8s 82ms/step - loss: 0.6045 - accuracy: 0.6983 - val_loss: 0.8446 - val_accuracy: 0.4730
Epoch 2/5
100/100 [==============================] - 8s 79ms/step - loss: 0.3470 - accuracy: 0.8570 - val_loss: 0.8395 - val_accuracy: 0.4730
Epoch 3/5
100/100 [==============================] - 8s 76ms/step - loss: 0.2520 - accuracy: 0.9020 - val_loss: 0.7280 - val_accuracy: 0.5520
Epoch 4/5
100/100 [==============================] - 7s 75ms/step - loss: 0.1875 - accuracy: 0.9267 - val_loss: 0.5860 - val_accuracy: 0.7090
Epoch 5/5
100/100 [==============================] - 8s 80ms/step - loss: 0.1408 - accuracy: 0.9450 - val_loss: 0.4099 - val_accuracy: 0.8770


In [175]:
model.evaluate(test_generator)

100/100 [==============================] - 1s 14ms/step


[0.05115674063563347, 0.859000027179718]